In [8]:
'''
A Multilayer Perceptron implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

'\nA Multilayer Perceptron implementation example using TensorFlow library.\nThis example is using the MNIST database of handwritten digits\n(http://yann.lecun.com/exdb/mnist/)\n\nAuthor: Aymeric Damien\nProject: https://github.com/aymericdamien/TensorFlow-Examples/\n'

In [9]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

import tensorflow as tf

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [10]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 2 # MNIST data input (img shape: 28*28)
n_classes = 2 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [11]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [12]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

In [13]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from tensorflow.python.framework import ops

n_samples = 1000
X, Y = datasets.make_moons(n_samples)
Y = np.eye(n_classes)[Y]
X_train, X_test, Y_train, Y_test = \
    train_test_split(X, Y, test_size=.4, random_state=42)
X = ops.convert_to_tensor(X_train, dtype="float")
Y = ops.convert_to_tensor(Y_train, dtype="float")
queue_x, queue_y = tf.train.slice_input_producer([X, Y],shuffle=False)
batch_x_op, batch_y_op = tf.train.batch([queue_x, queue_y], batch_size=batch_size, num_threads=1)

In [15]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(600/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = sess.run([batch_x_op, batch_y_op])
#             batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})
    coord.request_stop()
    coord.join(threads)

 Epoch: 0001 cost= 123.305864970
Epoch: 0002 cost= 33.610383670
Epoch: 0003 cost= 13.380857150
Epoch: 0004 cost= 11.713900407
Epoch: 0005 cost= 6.822565556
Epoch: 0006 cost= 3.938412547
Epoch: 0007 cost= 2.746650994
Epoch: 0008 cost= 1.861677180
Epoch: 0009 cost= 1.038926423
Epoch: 0010 cost= 0.405104553
Epoch: 0011 cost= 0.128045115
Epoch: 0012 cost= 0.017434297
Epoch: 0013 cost= 0.006680851
Epoch: 0014 cost= 0.002066042
Epoch: 0015 cost= 0.000329272
Optimization Finished!
Accuracy:

ValueError: Cannot feed value of shape (10000, 784) for Tensor u'Placeholder_2:0', which has shape '(?, 2)'

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer/input_producer/fraction_of_32_full/fraction_of_32_full_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_INT32], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](input_producer/input_producer/fraction_of_32_full/fraction_of_32_full, input_producer/input_producer/range)]]

Caused by op u'input_producer/input_producer/fraction_of_32_full/fraction_of_32_full_EnqueueMany', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
